### 0. Initializations

In [17]:
import csv
from math import log
import numpy as np
from array import array
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error
import math

policy_ID = 'PolicyNo' # name of the first column for identifying the policies

raw_train_data = '../data/cleaned_with_target.csv'
precleaned_train_data = '../../data/ver01/training_data_2016_cleaned_consolidated.csv'
target_data = '../data_part_3/target_data_2016.csv'

train_data = ''
test_data = ''

### 1. Reading the data into DataFrame

In [18]:
train_data_frame = pd.read_csv(precleaned_train_data)

In [19]:
target_data_frame = pd.read_csv(target_data)

In [20]:
raw_data_frame = pd.read_csv(raw_train_data)

### 2. Combine train and target in one complete frame

In [27]:
#complete_data_frame = pd.merge(left=train_data_frame,right=target_data_frame, left_on='species_id', right_on='species_id')
complete_data_frame = raw_data_frame #pd.merge(left=train_data_frame,right=target_data_frame) # the above example is for merging by column

#### Check which column is deleted 

In [28]:
np.setdiff1d(raw_data_frame.columns.values, complete_data_frame.columns.values)
#len(complete_data_frame.columns.values)
#len(raw_data_frame.columns.values)

array([], dtype=object)

In [29]:
df = complete_data_frame
#df = df.drop(policy_ID, 1) # will drop the policyID column from the dataframe
cols = list(df.columns.values) # the argument return the header
cols.remove(policy_ID)
#print cols
rem_dup=df.drop_duplicates(cols)
df.shape

(424431, 68)

### 3. Removing Outliers

In [30]:
#def remove_outliers(df, column, min_val, max_val):
#    col_values = df[column].values
#    df[column] = np.where(np.logical_or(col_values<=min_val, col_values>=max_val), np.NaN, col_values)
    
#    return df 

#df_all = remove_outliers(df, column='Loss_Ratio', min_val=-1, max_val=10000)
#df.shape

df = df[df['Loss_Ratio'] < 10000]

df = df[df['Loss_Ratio'] == 0]
df.shape
complete_data_frame = df
df.shape

(405564, 68)

In [14]:
features = ['Driver_Total','Driver_Total_Male', 'Driver_Total_Female', 'Driver_Total_Single', 'Driver_Total_Married','Driver_Minimum_Age', 'Driver_Maximum_Age', 'Driver_Total_Teenager_Age_15_19', 'Driver_Total_College_Ages_20_23', 'Driver_Total_Young_Adult_Ages_24_29']#, 'Driver_Total_Low_Middle_Adult_Ages_30_39', 'Driver_Total_Middle_Adult_Ages_40_49', 'Driver_Total_Adult_Ages_50_64', 'Driver_Total_Senior_Ages_65_69', 'Driver_Total_Upper_Senior_Ages_70_plus']
all_target = ['Claim_Count', 'Loss_Amount', 'Frequency', 'Severity', 'Loss_Ratio']
all_features = list(set(list(df.columns.values)) - set(all_target))
print all_features
target_1 = ['Loss_Amount']
target_2 = ['Loss_Ratio']

['Policy_Method_Of_Payment', 'Vehicle_Annual_Miles', 'Vehicle_Youthful_Driver_Training_Code', 'Driver_Total_Married', 'Driver_Total_Related_To_Insured_Child', 'Vehicle_Youthful_Good_Student_Code', 'SYS_Renewed', 'Driver_Total_Teenager_Age_15_19', 'Vehicle_Age_In_Years', 'Vehicle_Passive_Restraint', 'Driver_Total_Female', 'Vehicle_Driver_Points', 'Vehicle_Comprehensive_Coverage_Limit', 'Driver_Total_Low_Middle_Adult_Ages_30_39', 'Policy_Company', 'Driver_Total_Senior_Ages_65_69', 'Vehicle_Make_Description', 'Vehicle_New_Cost_Amount', 'Vehicle_Miles_To_Work', 'Driver_Total_Single', 'Vehicle_Collision_Coverage_Deductible', 'EEA_Liability_Coverage_Only_Indicator', 'Vehicle_Collision_Coverage_Indicator', 'Driver_Total_Related_To_Insured_Spouse', 'Vehicle_Anti_Theft_Device', 'Policy_Billing_Code', 'Vehicle_Number_Of_Drivers_Assigned', 'Policy_Reinstatement_Fee_Indicator', 'Driver_Total_Middle_Adult_Ages_40_49', 'SYS_New_Business', 'Vehicle_Make_Year', 'Driver_Total_Licensed_In_State', 'Drive

In [31]:
train_data,test_data = train_test_split(complete_data_frame, test_size = 0.2, random_state= 5)

### 5. Feature Selection

In [32]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
features = f_regression(df[all_features], df[target_2])

ValueError: could not convert string to float: CHEV TRAILBLAZE  

### Trying to apply regression algorithm on the data for some of the features 

In [ ]:
regr = linear_model.LinearRegression()
X = train_data[features].values
y = train_data[target_2].values
#X = X.reshape(-1,1)
#print X
regr.fit(X, y)

In [ ]:
print test_data[target_2].mean()
X_test = test_data[features].values
#X_test = X_test.reshape(-1,1)
y_test = test_data[target_2].values
#print("Mean squared error: %.2f"
 #     % np.mean((regr.predict(diabetes_X_test) - diabetes_y_test) ** 2))
rmse = math.sqrt(mean_squared_error(y_test, regr.predict(X_test))) # y_test -> true, X_test <- predicted
print('RMSE: %.2f' % rmse)
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(X_test, y_test))
regr.score(X_test, y_test)

## Cross-Validation

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(regr, X, y, cv=2)
scores
scores.mean()

### Accuracy metric
http://scikit-learn.org/stable/modules/cross_validation.html#obtaining-predictions-by-cross-validation

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
predicted = cross_val_predict(regr, X, y, cv=10)
metrics.accuracy_score(y, predicted) 